<a href="https://colab.research.google.com/github/SinaRampe/pubmed_downloader/blob/main/pubmed_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install metapub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.3/120.3 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━

In [71]:
import pandas as pd
from metapub import PubMedFetcher
from metapub import FindIt
import requests
import shutil
import os

In [6]:
keyword="cpic"
num_of_articles=10

In [87]:
save_directory = f"pubmed_{keyword}"

if not os.path.exists(save_directory):
    os.makedirs(save_directory)

In [88]:
fetch = PubMedFetcher()

In [89]:
pmids = fetch.pmids_for_query(keyword, retmax=num_of_articles)

In [90]:
collected_data = []

In [91]:
for pmid in pmids:

  article = fetch.article_by_pmid(pmid)

  data = {}

  data["PMID"] = pmid
  data["Title"] = article.title
  data["Abstract"] = article.abstract
  data["Author"] = article.authors
  data["Year"] = article.year
  data["Volume"] = article.volume
  data["Issue"] = article.issue
  data["Journal"] = article.journal
  data["Citation"] = article.citation
  data["Link"] = "https://pubmed.ncbi.nlm.nih.gov/"+pmid+"/"

  collected_data.append(data) 



In [92]:
df = pd.DataFrame(collected_data)

In [93]:
df.head(2)

,PMID,Title,Abstract,Author,Year,Volume,Issue,Journal,Citation,Link
0,37155913,The Contribution of Pharmacogenetic Drug Inter...,Context: Clinical Pharmacogenetics Implementat...,"[David S, Meltzer D, Singh L, Hulick P, Pruitt...",2023,None,21(Suppl 1),Ann Fam Med,"David S, et al. The Contribution of Pharmacoge...",https://pubmed.ncbi.nlm.nih.gov/37155913/
1,37127518,The Prevalence and Features of Medications Wit...,BACKGROUND: Genetic variants are associated wi...,"[Chang YL, Hsiao TH, Wu MF, Chen CH]",2023,None,None,Transplant Proc,"Chang YL, et al. The Prevalence and Features o...",https://pubmed.ncbi.nlm.nih.gov/37127518/


In [94]:
pdf_url_or_reason = []
pdf_url = []

In [95]:
for pmid in pmids:
  src = FindIt(pmid)
  if src.url is None:
    pdf_url_or_reason.append(src.reason)
  else: 
    pdf_url_or_reason.append(src.url)
    pdf_url.append((pmid, src.url))

In [96]:
df["pdf-url"] = pdf_url_or_reason

In [97]:
df.to_csv('pubmed_articles.csv')

In [98]:
for pmid, url in pdf_url:

  response = requests.get(url, allow_redirects=True)

  with open(f"{pmid}.pdf", 'wb') as f:
    f.write(response.content)

    

other resources for ideas how to extend my code

https://github.com/billgreenwald/Pubmed-Batch-Download

https://github.com/ddomingof/PubMed2PDF